In [6]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd 
import random
import os  
from selenium.webdriver.common.keys import Keys

In [7]:
#1. 사용자 인터페이스 설정

print('='*100)
print('수비를 위한 네이버 웹소설 댓글 및 평점 크롤러')
print('-'*100)
print('이 크롤러는 1화부터 등록된 마지막 화 까지 댓글 및 평점을 순차 수집합니다')
print('='*100)
nov_title = input('1. 댓글 및 평점 정보 수집하고 싶은 웹소설 제목을 입력하세요 : ')

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

f_dir = input('''2. 크롤링 결과물 파일 저장할 문서 경로를 폴더 이름까지만 입력해주세요(예 : /Users/kibeomkim/Desktop/folder2) : 
모르겠으면 엔터 키를 눌러주세요. 기본 경로로 저장됩니다. ''')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터'
os.makedirs(f_dir+'/'+s+'-'+nov_title)
os.chdir(f_dir+'/'+s+'-'+nov_title)

#2. 파일 확장자 & 경로 미리 변수로 담아두기 
f_name = s+'-'+nov_title+'.txt'

fc_name = s+'-'+nov_title+'.csv'
fc_name2 = s+'-'+nov_title+'댓글정보'+'.csv'

fx_name = s+'-'+nov_title+'.xlsx'
fx_name2 = s+'-'+nov_title+'댓글정보'+'.xlsx'
s_time = time.time()

#3. 웹드라이버 호출 & 네이버 웹소설 페이지로 이동하기
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR')
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

driver = webdriver.Chrome(path, options=options)
driver.get('https://novel.naver.com/webnovel/weekday.nhn')
time.sleep(random.randrange(2, 5))


#4. 웹소설 검색창 찾고 검색어 넣기 + 엔터
element1 = driver.find_element_by_xpath('/html/body/div/div[2]/div/div[1]/div[2]/span/input')
element1.send_keys(nov_title)
element1.send_keys('\n')
time.sleep(random.randrange(3, 5))


url_list = []
no0 = 0
#5. 검색어로 조회된 웹소설 모두 보여주기 
html0 = driver.page_source 
soup0 = BeautifulSoup(html0, 'html.parser')

try : 
    li_list = soup0.find('div', 'srch_cont NE=a:all').find_all('li')
    number = len(li_list)
    print('%s 키워드로 총 %s 개 웹소설이 발견되었습니다.'%(nov_title, number))
except : 
    print('%s 키워드로 검색된 웹소설이 없습니다.')
    driver.quit()
    
for a in li_list : 
    each_title = a.find('span', 'ellipsis')
    real_title = each_title.get_text().replace('\n', "").strip()
    print('%s. %s' %(no0, real_title))
    url = a.find('a')['href']
    url_list.append(url)
    no0 += 1
try : 
    choice = int(input('0번~%s 번 중 댓글 및 평점 수집할 웹소설 번호를 하나만 입력하세요 : '))
except : 
    print('번호를 하나만 입력하세요')


driver.quit()

path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
driver = webdriver.Chrome(path, options=options)
driver.get('https://novel.naver.com'+url_list[choice])
time.sleep(random.randrange(2, 5))



수비를 위한 네이버 웹소설 댓글 및 평점 크롤러
----------------------------------------------------------------------------------------------------
이 크롤러는 1화부터 등록된 마지막 화 까지 댓글 및 평점을 순차 수집합니다
1. 댓글 및 평점 정보 수집하고 싶은 웹소설 제목을 입력하세요 : 실베스터 공녀
2. 크롤링 결과물 파일 저장할 문서 경로를 폴더 이름까지만 입력해주세요(예 : /Users/kibeomkim/Desktop/folder2) : 
모르겠으면 엔터 키를 눌러주세요. 기본 경로로 저장됩니다. 
실베스터 공녀 키워드로 총 1 개 웹소설이 발견되었습니다.
0. 황태자의 보좌관 실베스터 공녀
0번~%s 번 중 댓글 및 평점 수집할 웹소설 번호를 하나만 입력하세요 : 0


In [8]:
html44 = driver.page_source
soup44 = BeautifulSoup(html44, 'html.parser')
text44 = soup44.find('h3', 'tit_num_list').find('span','total').get_text()
text45 = int(text44.replace('(', "").replace(")", "").strip())
print('총 %s 회차가 존재합니다' %text45)

총 83 회차가 존재합니다


In [9]:
#1화부터 보기 클릭
try : 
    driver.find_element_by_xpath('/html/body/div/div[3]/div/div[2]/div[1]/h3/span[3]/label[2]').click()
    time.sleep(3)
except : 
    print("")

In [10]:
#7. 평점 수집하기 
star_grade1 = []
star_people1 = []
heart1 = []
nicknamelist1 = []
contentslist1 = []
datelist1 = []
gogamlist1 = []
bgogamlist1 = []
h1 = []
h3 = []

no22 = 2
f = open(f_name, 'a', encoding='utf-8')
no11 = 1
no33 = 1
no45 = 1
no55 = 1
no66 = 0

while no22 == 2 : 
    html23 = driver.page_source 
    soup23 = BeautifulSoup(html23, 'html.parser')
    li_number = len(soup23.find('ul').find_all('li'))
    
    for x in range(1, 11) : 
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div[2]/div[1]/ul/li[%s]/a/div/p[1]' %x).click()
        time.sleep(2)
        
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(1)
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_UP)
        time.sleep(3)

        html1 = driver.page_source 
        soup1 = BeautifulSoup(html1, 'html.parser')
        
        driver.switch_to.frame('nCommentIframe')
        html88 = driver.page_source
        soup88 = BeautifulSoup(html88, 'html.parser')
        ddat_number = int(soup88.find('span', 'u_cbox_count').get_text().strip())

        
        
    
        #8. 댓글 수집하기 
        while no22 == 2 : 
            no66 += 1 
            html3 = driver.page_source
            soup3 = BeautifulSoup(html3, 'html.parser')
            cbox_list = soup3.find('ul', 'u_cbox_list').find_all('div', 'u_cbox_area')
            print(len(cbox_list))
            time.sleep(3)
        
            for b in cbox_list : 
                contents = b.find('div', 'u_cbox_text_wrap').find('span', 'u_cbox_contents').get_text().strip()
                contentslist1.append(contents)
                print('댓글 내용 :',contents)
                f.write('댓글 내용 :'+contents+'\n')
                #time.sleep(1)
                

                    
            try : 
                if no55 == 1 : 
                    driver.find_element_by_xpath('/html/body/div/div/div[5]/div/a[%s]'%no55).click()
                    no55 = no55 + 2
                    time.sleep(2)
                elif no55 != 1 : 
                    if no66 == 11 : 
                        no55 = 3
                    if no55 == 13 : 
                        no55 = 3
                    driver.find_element_by_xpath('/html/body/div/div/div[5]/div/a[%s]' %no55).click()
                    no55 += 1
                    time.sleep(2)
            except : 
                print('조회 가능한 모든 댓글을 조회했습니다')
                print('\n')
                break
                
        print('%s화 평점 및 댓글 수집 완료 ========' %no11)
        print('\n')
        driver.back()
        no55 = 1
        no66 = 0
        time.sleep(4)
        no11 += 1
        if no11 > text45 : 
            break
    try : 
        driver.find_element_by_xpath('/html/body/div/div[3]/div/div[2]/div[1]/div[3]/a[%s]' %no33).click()
        no33 += 1
    except : 
        print('가능한 모든 회차를 조회하였습니다')
        break

f.close()
driver.quit()
e_time = time.time()
total_time = e_time - s_time

ddubi = pd.DataFrame()
ddub2 = pd.DataFrame()

ddub2['댓글 내용'] = contentslist1



print('='*100)
print('%s 웹소설, 총 %s 회차의 댓글 및 평점을 수집했습니다' %(nov_title, no11))
print('총 소요시간은 %s 초 입니다' %round(total_time, 1))
print('='*100)



4
댓글 내용 : 없었데 - 없었대  입니당 글은 재밌네용
댓글 내용 : 정주행 시작!
댓글 내용 : 즐독하고 갑니다
댓글 내용 : 재밌어요 ㅎㅎㅎ
조회 가능한 모든 댓글을 조회했습니다


1화 평점 및 댓글 수집 완료 ========


1
댓글 내용 : 누가 감히 여주를 건드리려해?!
조회 가능한 모든 댓글을 조회했습니다


2화 평점 및 댓글 수집 완료 ========


2
댓글 내용 : 
댓글 내용 : 소설속 여주들은 비정상이 많아. 어딜 만져?
조회 가능한 모든 댓글을 조회했습니다


3화 평점 및 댓글 수집 완료 ========


2
댓글 내용 : 
댓글 내용 : 정보를 얻는 방법이 좀 허술하네요.
조회 가능한 모든 댓글을 조회했습니다


4화 평점 및 댓글 수집 완료 ========


1
댓글 내용 : 
조회 가능한 모든 댓글을 조회했습니다


5화 평점 및 댓글 수집 완료 ========


1
댓글 내용 : 
조회 가능한 모든 댓글을 조회했습니다


6화 평점 및 댓글 수집 완료 ========


0
조회 가능한 모든 댓글을 조회했습니다


7화 평점 및 댓글 수집 완료 ========


1
댓글 내용 : 
조회 가능한 모든 댓글을 조회했습니다


8화 평점 및 댓글 수집 완료 ========


1
댓글 내용 : 사건 스토리가 빨라서 좋긴 한데요.. 감정선도 아주 빠르네요.금사빠 남주에 어화둥둥 여주인건가봐요. 개연성이 부족하다고 생각합니다.
조회 가능한 모든 댓글을 조회했습니다


9화 평점 및 댓글 수집 완료 ========


1
댓글 내용 : 
조회 가능한 모든 댓글을 조회했습니다


10화 평점 및 댓글 수집 완료 ========


1
댓글 내용 : 
조회 가능한 모든 댓글을 조회했습니다


11화 평점 및 댓글 수집 완료 ========


2
댓글 내용 : 
댓글 내용 : "집으로 돌아온 에밀리는 의자에서 책을 읽고 있던 아벨을 보고는 그의 앞자리에 앉아 그를 말없이 바라보기 시작했다."  아벨이 에밀리랑 사나요?? 오

댓글 내용 : 
조회 가능한 모든 댓글을 조회했습니다


80화 평점 및 댓글 수집 완료 ========


1
댓글 내용 : 오... 셀리나의 능력 발현!
조회 가능한 모든 댓글을 조회했습니다


81화 평점 및 댓글 수집 완료 ========


0
조회 가능한 모든 댓글을 조회했습니다


82화 평점 및 댓글 수집 완료 ========


1
댓글 내용 : 작가님 힘내시고 건강도 챙기셔요~
조회 가능한 모든 댓글을 조회했습니다


83화 평점 및 댓글 수집 완료 ========


가능한 모든 회차를 조회하였습니다
실베스터 공녀 웹소설, 총 84 회차의 댓글 및 평점을 수집했습니다
총 소요시간은 1202.8 초 입니다


list88 = []
for d in list88 : 
    print(d) # for 반복문을 [] 빈 리스트에서 돌리면 실행은 된다. 근데 아무 결과값도 내놓지 않는다.왜? 
print('a') #--> for '반복문'이 아니라 for '조건문'이다. 
# --> for d in list88 매칭할 조건이 안 되니까,
# --> for 코드 블록을 통째로 그냥 무시한다.  그래서 아무 결과값도 내놓지 않는거다. 
# while도 엄밀히 말하면 조건문이다. 